In [1]:
import pandas as pd
from sqlalchemy import create_engine
from prefect import flow,task

#Load the raw data
dataframe = pd.read_csv('supermarket_sales.csv')
dataframe.head()


# Snowflake connection details
user = 'BreezyDataAnalyst'
password = 'joxxed-zinsyg-Casje4'
account = 'uqviiry-ir83823'
warehouse = 'your_warehouse'
database = 'SUPERMARKET'
schema = 'PUBLIC'
role = 'ACCOUNTADMIN'

In [2]:
#Cleaning data
@task
def cleaning_data(df):
    #change the date column to YYYY-MM-DD format
    df['Date'] = pd.to_datetime(df['Date'], dayfirst=True).dt.strftime('%Y-%m-%d')
    #Change columns names
    df=df.rename(columns={'Invoice ID':'Invoice_ID','Customer type':'Customer_type','Product line':'Product_line','Unit price':'Unit_price','Tax 5%':'Tax_5%','gross margin percentage':'gross_margin_%',
    'gross income':'gross_income'})
    return df

In [3]:
#Send data to Snowflake
@task
def send_data_snowflake(df):
    # Create a connection engine
    engine = create_engine(f'snowflake://{user}:{password}@{account}/{database}/{schema}?role={role}')
    # Write the DataFrame to Snowflake
    df.to_sql('Sales', con=engine, index=False, if_exists='replace')
    print("Data sent to Snowflake successfully!")

In [4]:
# Send the cleaned data to Snowflake

@flow
def data_pipeline():
    #Cleanig data
    df=cleaning_data(dataframe)
    #Send data to snowflake
    send_data_snowflake(df)
    
if __name__ == "__main__":
    data_pipeline.serve(name='ETL_deployment')

11:24:33.696 | INFO    | prefect.engine - Created flow run 'unnatural-dragon' for flow 'data-pipeline'

11:24:33.704 | INFO    | prefect.engine - View at http://127.0.0.1:4200/runs/flow-run/c9a8cd6f-6e9b-4315-8f5b-3627bfc22922

11:24:33.953 | INFO    | Task run 'cleaning_data-a4c' - Created task run 'cleaning_data-a4c' for task 'cleaning_data'

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_6376\398418831.py:5: UserWarning: Parsing dates in MM/DD/YYYY format when dayfirst=True was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['Date'] = pd.to_datetime(df['Date'], dayfirst=True).dt.strftime('%Y-%m-%d')


11:24:33.987 | INFO    | Task run 'cleaning_data-a4c' - Finished in state Completed()

11:24:34.020 | INFO    | Task run 'send_data_snowflake-538' - Created task run 'send_data_snowflake-538' for task 'send_data_snowflake'

C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\snowflake\sqlalchemy\base.py:1068: SAWarning: The GenericFunction 'flatten' is already registered and is going to be overridden.
  functions.register_function("flatten", flatten)
C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\snowflake\connector\config_manager.py:351: UserWarning: Bad owner or permissions on C:\Users\Lenovo\.snowflake\connections.toml
  warn(f"Bad owner or permissions on {str(filep)}{chmod_message}")


Data sent to Snowflake successfully!


11:24:54.652 | INFO    | Task run 'send_data_snowflake-538' - Finished in state Completed()

11:24:54.781 | INFO    | Flow run 'unnatural-dragon' - Finished in state Completed()